In [2]:
from keras.layers import Dense,LSTM,Embedding
from keras.utils import to_categorical
from keras.models import Sequential
from sklearn.cross_validation import train_test_split
import numpy as np
from gensim.models import Word2Vec
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model

file=open('kafka.txt')
string=file.read()
words=string.split()

sentences=[words[i:i+5] for i in range(5000)]
targets=[words[i+5] for i in range(5000)]


vocab=list(set(words[:5005]))

words_to_code=dict((i,j) for j,i in enumerate(vocab))
code_to_words=dict((i,j) for i,j in enumerate(vocab))
newtargets=[words_to_code[w] for w in targets]
for i in range(5000):
    for j in range(5):
        sentences[i][j]=words_to_code[sentences[i][j]]
sentences=np.array(sentences)



newtargets=to_categorical(newtargets,num_classes=len(vocab))
newtargets=np.array(newtargets)

model=Sequential()
model.add(Embedding(len(vocab),40,input_length=5))
model.add(LSTM(40,dropout=0.3,input_shape=(5,40)))
model.add(Dense(len(vocab),activation='softmax'))
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])

X_train,X_test,Y_train,Y_test=train_test_split(sentences,newtargets)

/Users/harshtiku/Desktop/exp/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/Users/harshtiku/Desktop/exp/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
history=model.fit(X_train,Y_train,epochs=10,)
model.save('newtextmodel.h5')

Epoch 1/10
3750/3750 [==============================] - 4s 960us/step - loss: 5.5375 - acc: 0.0536
Epoch 2/10
3750/3750 [==============================] - 4s 938us/step - loss: 5.4902 - acc: 0.0552
Epoch 3/10
3750/3750 [==============================] - 4s 944us/step - loss: 5.4443 - acc: 0.0600
Epoch 4/10
3750/3750 [==============================] - 4s 982us/step - loss: 5.4067 - acc: 0.0600
Epoch 5/10
3750/3750 [==============================] - 4s 939us/step - loss: 5.3668 - acc: 0.0632
Epoch 6/10
3750/3750 [==============================] - 4s 948us/step - loss: 5.3218 - acc: 0.0637
Epoch 7/10
3750/3750 [==============================] - 4s 949us/step - loss: 5.2864 - acc: 0.0659
Epoch 8/10
3750/3750 [==============================] - 4s 951us/step - loss: 5.2445 - acc: 0.0683
Epoch 9/10
3750/3750 [==============================] - 4s 958us/step - loss: 5.2054 - acc: 0.0667
Epoch 10/10
3750/3750 [==============================] - 4s 985us/step - loss: 5.1650 - acc: 0.0739


In [ ]:
?model.fit

In [5]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.show()


<Figure size 640x480 with 1 Axes>

In [6]:
def generate_test():
    import time
    sentence='His room, a proper human'
    sentence=sentence.split()
    sentence=[words_to_code[w] for w in sentence]
    sentence=np.array(sentence)
    sentence=sentence.reshape(1,5)
    
    model=load_model('newtextmodel.h5')
    
    while True:
        x=np.argmax(model.predict(sentence))
        print(code_to_words[x],end=' ')
        sentence=sentence.flatten()
        sentence=sentence.tolist()
        sentence.append(x)
        sentence=sentence[1:]
        sentence=np.array(sentence)
        sentence=sentence.reshape(1,5)
        time.sleep(1)

In [7]:
generate_test()

clerk His a chief clerk His His His a clerk His His His His His His His His His His His His His His His 

KeyboardInterrupt: 